In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import Audio, display
import os
import pandas as pd
import random
from pathlib import Path
import sys

project_root = Path().resolve().parent
sys.path.append(str(project_root))

from misophonia_dataset.interface import DEFAULT_DIR
from misophonia_dataset.miso_dataset import FSD50K, FOAMS, ESC50, MisophoniaData

In [3]:
print("Before: ", os.getcwd())
binamix_dir = "/workspaces/misophonia-dataset/Binamix"
if os.getcwd() != binamix_dir:
    os.chdir("/workspaces/misophonia-dataset/Binamix")

print("After: ", os.getcwd())
from binamix.sadie_utilities import *

Before:  /workspaces/misophonia-dataset
After:  /workspaces/misophonia-dataset/Binamix


In [9]:
esc50_mapping = Path("../data/mappings/esc50_to_foams_mapping.json")

fsd50k_mapping = Path("../data/mappings/fsd50k_to_foams_mapping.json")
background_classes = Path("../data/mappings/background_classes.json")

esc50 = ESC50(mapping=esc50_mapping, save_dir=DEFAULT_DIR)
fsd50k = FSD50K(mapping=fsd50k_mapping, backgrounds=background_classes, save_dir=DEFAULT_DIR)
foams = FOAMS(save_dir = DEFAULT_DIR)
# fsd50_eval = FSD50KEval(mapping=fsd50k_mapping, backgrounds=background_classes, save_dir=DEFAULT_DIR)
miso_data = MisophoniaData(source_data = [esc50, fsd50k, foams])

ESC50 dataset has already been downloaded and unzipped at ../data
Filtering trigger samples from ESC50...
Creating train/valid/test split...
FSD50K dataset has already been downloaded and unzipped at ../data
Creating train/valid/test split...
FOAMS dataset has already been downloaded and unzipped.
Metadata has already been downloaded.
Creating train/valid/test split...


In [23]:
print(miso_data.train.shape)

print(miso_data.train[miso_data.train["isTrig"] == 2].shape)
print(miso_data.train[miso_data.train["isTrig"] == 0].shape)
print(miso_data.train[miso_data.train["isTrig"] == 1].shape)

print(miso_data.val[miso_data.val["isTrig"] == 2].shape)
print(miso_data.val[miso_data.val["isTrig"] == 0].shape)
print(miso_data.val[miso_data.val["isTrig"] == 1].shape)

(20182, 5)
(18070, 5)
(621, 5)
(1491, 5)
(4442, 5)
(145, 5)
(380, 5)


In [ ]:
k = 5
t_indices = random.sample(range(0, t_meta.shape[0]), k)
c_indices = random.sample(range(0, c_meta.shape[0]), k)
b_indices = random.sample(range(0, b_meta.shape[0]), k)

t_sample = [int(t_meta.iloc[t_indices[i]]["fname"]) for i in range(k)]
c_sample = [int(c_meta.iloc[c_indices[i]]["fname"]) for i in range(k)]
b_sample = [int(b_meta.iloc[b_indices[i]]["fname"]) for i in range(k)]

In [ ]:
def binaural_mix(trigger_file: str, control_file: str, background_file: str):
    """
    Takes a trigger sound, control sound, background sound, and creates a binaural soundtrack of trigger + background and control + background.
    Saves binaural mixes as .wav files. Saves both to a specified folder
    Params:
        trigger_file: trigger .wav file
        control_file: control .wav file
        background_file: background .wav file
    Returns:
        None
    """
    # MIXING PARAMS
    subject_id = "D2" 
    ir_type = "HRIR" # Options HRIR, BRIR
    speaker_layout = "none"
    sr = 44100
    duration = min(b_meta.loc[background_file]["Duration"], 
                t_meta.loc[trigger_file]["Duration"],
                c_meta.loc[control_file]["Duration"]
                )

    # FILE PATHS
    trigger_path = "../data/FSD50K/" + str(trigger_file) + ".wav"
    background_path = "../data/backgrounds_dev/" + str(background_file) + ".wav"
    control_path = "../data/FSD50K/controls/" + str(control_file) + ".wav"

    # MIXING
    trigger, _ = librosa.load(trigger_path, sr=sr, mono=True, duration=duration)
    control, _ = librosa.load(control_path, sr=sr, mono=True, duration=duration)
    background, _ = librosa.load(background_path, sr=sr, mono=True, duration=duration)

    azimuth = random.randint(-180, 180)
    elevation = random.randint(-180, 180)

    track1 = TrackObject(name="trigger", azimuth=azimuth, elevation=elevation, level=0.6, reverb=0.0, audio=trigger)
    track2 = TrackObject(name="control", azimuth=azimuth, elevation=elevation, level=0.6, reverb=0.0, audio=control)
    track3 = TrackObject(name="background", azimuth=0, elevation=0, level=0.6, reverb=0.0, audio=background)

    trigger_mix = mix_tracks_binaural([track1, track3], subject_id, sr, ir_type, speaker_layout, mode="auto", reverb_type="1")
    control_mix = mix_tracks_binaural([track2, track3], subject_id, sr, ir_type, speaker_layout, mode="auto", reverb_type="1")

    #DISPLAY
    display(Audio(trigger_mix, rate=sr))
    display(Audio(control_mix, rate=sr))
    return duration

In [ ]:
binaural_mix(trigger_file=t_sample[0], control_file=c_sample[0], background_file=b_sample[0])

In [ ]:
binaural_mix(trigger_file=t_sample[1], control_file=c_sample[1], background_file=b_sample[1])

In [ ]:
binaural_mix(trigger_file=t_sample[2], control_file=c_sample[2], background_file=b_sample[2])

In [ ]:
binaural_mix(trigger_file=t_sample[3], control_file=c_sample[3], background_file=b_sample[3])

In [ ]:
binaural_mix(trigger_file=t_sample[4], control_file=c_sample[4], background_file=b_sample[4])